In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,48486
2,Huelva,Confirmados PDIA 14 días,1268
3,Huelva,Tasa PDIA 14 días,"247,09160706978196"
4,Huelva,Confirmados PDIA 7 días,679
5,Huelva,Tasa PDIA 7 dias,"132,31482744509617"
6,Huelva,Total Confirmados,48693
7,Huelva,Curados,45950
8,Huelva,Fallecidos,423


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  48486.0


/tmp/ipykernel_19220/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13504.0


/tmp/ipykernel_19220/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_19220/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_19220/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_19220/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 639 personas en los últimos 7 días 

Un positivo PCR cada 331 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,48486.0,679.0,1268.0,513170.0,132.314827,247.091607,105.0
Huelva-Costa,28623.0,371.0,700.0,289548.0,128.130742,241.756116,60.0
Huelva (capital),13504.0,225.0,435.0,143837.0,156.427067,302.425662,38.0
Sierra de Huelva-Andévalo Central,4312.0,104.0,157.0,67391.0,154.323278,232.968794,26.0
Condado-Campiña,15121.0,202.0,409.0,156231.0,129.295722,261.791834,19.0
Aracena,713.0,32.0,36.0,8255.0,387.643852,436.099334,17.0
Cumbres Mayores,149.0,11.0,14.0,1749.0,628.930818,800.457404,7.0
Aljaraque,1744.0,38.0,66.0,21474.0,176.958182,307.348421,6.0
Bollullos Par del Condado,1412.0,32.0,71.0,14387.0,222.423021,493.501077,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),18.0,1.0,5.0,243.0,411.522634,2057.613169,0.0
Puerto Moral,10.0,5.0,5.0,278.0,1798.561151,1798.561151,0.0
Paterna del Campo,280.0,11.0,50.0,3457.0,318.194967,1446.340758,0.0
Cumbres Mayores,149.0,11.0,14.0,1749.0,628.930818,800.457404,7.0
Corteconcepción,50.0,3.0,4.0,536.0,559.701493,746.268657,0.0
Bollullos Par del Condado,1412.0,32.0,71.0,14387.0,222.423021,493.501077,5.0
Palos de la Frontera,1213.0,31.0,53.0,11742.0,264.009538,451.371146,3.0
Aracena,713.0,32.0,36.0,8255.0,387.643852,436.099334,17.0
Valverde del Camino,926.0,36.0,55.0,12750.0,282.352941,431.372549,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Granada de Río-Tinto (La),18.0,1.0,5.0,243.0,411.522634,2057.613169,0.0,0.200000
Escacena del Campo,240.0,1.0,5.0,2287.0,43.725404,218.627022,0.0,0.200000
Paterna del Campo,280.0,11.0,50.0,3457.0,318.194967,1446.340758,0.0,0.220000
Higuera de la Sierra,75.0,1.0,4.0,1291.0,77.459334,309.837335,0.0,0.250000
Rociana del Condado,699.0,6.0,21.0,7939.0,75.576269,264.516942,2.0,0.285714
San Juan del Puerto,781.0,3.0,9.0,9411.0,31.877590,95.632770,0.0,0.333333
Villarrasa,236.0,1.0,3.0,2211.0,45.228403,135.685210,0.0,0.333333
Ayamonte,2135.0,6.0,17.0,21104.0,28.430629,80.553450,2.0,0.352941
Cortegana,309.0,5.0,13.0,4602.0,108.648414,282.485876,0.0,0.384615
